# pyModis notebook

## Download data

First we have to import pymodis library and set all the needed variables

In [1]:
import os
import glob
from IPython.core.display import Image
from pymodis import downmodis

WxPython missing, no GUI enabled


In [2]:
# product
product = 'MOD16A2.006' # anual ET gap filled v6
# destination folder
dest = 'F:/OneDrive - Universidad de Cantabria/Cartografia/MODIS/' + product.split('.')[0] + '/'
# tiles to download
tiles = 'h17v04'#, 'h17v05']
# starting day
start = '2001.01.01'
# ending day
end = '2019.10.24'
# number of day to download
delta = 24

In [3]:
modisDown = downmodis.downModis(destinationFolder=dest,
                                password='Chomolungma1618', user='casadoj',
                                url='https://e4ftl01.cr.usgs.gov', product=product, 
                                tiles=tiles, today=end, delta=0)

In [4]:
modisDown.connect()

In [5]:
modisDown.downloadsAllDay()

At this point we can initialize the downmodis object. We are going to download MODIS LST product, in the "/tmp" directory

Hace falta tener en la carpeta `C:\Users\username\` el archivo `.netrc`, que contiene lo siguiente:
> machine urs.earthdata.nasa.gov<br>
> login username<br>
> password password<br>

La url donde se encuentran los datos de MODIS es https://e4ftl01.cr.usgs.gov. Los datos del satélite Terra corresponden a _MOLT_ y los de Aqua a _MOLA_.

In [ ]:
import datetime

In [ ]:
dates = []
for year in np.arange(2001, 2020):
    for day in range(1, 366, 8):
    #     print(day)
        date = datetime.datetime(year - 1, 12, 31) + datetime.timedelta(day)
        if date > datetime.datetime(2019, 11, 1):
            break
        dates.append(date.strftime("%Y.%m.%d"))

In [ ]:
dates

In [ ]:
dates[-1] - dates[0]

In [ ]:
for day in dates[:2]:
    modisDown = downmodis.downModis(destinationFolder=dest,
                                    password='Chomolungma1618', user='casadoj',
                                    url='https://e4ftl01.cr.usgs.gov', tiles=tiles,
                                    product=product, today=day, delta=0)
    modisDown.connect()
    modisDown.downloadsAllDay()

At this point we start the download of files.

<div style="border: 1px solid red;"><h6 style="color:#ff4040; margin: 0.5em;">Warning</h6>
<span style="margin: 0.5em;">The next operation could take some minutes depending on your internet speed</span></div>

In [ ]:
modisDown.downloadsAllDay()

In [ ]:
# create the list of files to use
files = glob.glob(os.path.join(dest, product.split('.')[0] + '*.hdf'))
print(files)

In [ ]:
from topng import hdf2png
pngfile = hdf2png(files[0])
Image(filename=pngfile)

## Parse the data

We import the needed modules and after we obtain a list with the full path to the downloaded MODIS files

In [ ]:
from pymodis import parsemodis

#### Parse single data

At this point we can parse one file (in this case the first one), creating a parseModis object

In [ ]:
modisParse = parsemodis.parseModis(files[0])

Now you can obtain several parameters from the MODIS metadata, we query only some values to show you:

In [ ]:
# bounding box of the tiles
modisParse.retBoundary()

In [ ]:
# quality statistics
modisParse.retMeasure()

In [ ]:
modisParse.retRangeTime()

Check parseModis class documentation for more info

In [ ]:
parsemodis.parseModis?

#### Parse multiple data

After the test with only a MODIS file, we are going to test the parsing for multiple files

In [ ]:
modisMultiParse = parsemodis.parseModisMulti(files)

Now you can obtain the value of boundary

In [ ]:
modisMultiParse.valBound()
modisMultiParse.boundary

or write a xml for a MODIS mosaic file

In [ ]:
modisMultiParse.writexml(os.path.join(dest, 'modismultiparse.xml'))

at the end you can read the created file and print the lines

In [ ]:
f = open(os.path.join(dest, 'modismultiparse.xml'))
lines = f.readlines()
p = [l.strip() for l in lines]
f.close()
print "\n".join(p)

## Create a mosaic file

We are going to use the GDAL library to create the mosaic. As subset we are going to use daily and nightly temperature and it's quality layer

In [ ]:
from pymodis.convertmodis_gdal import createMosaicGDAL
# [daily temp, quality for daily, not used, not used, nightly temp, quality for nightly]
subset = [1,1,0,0,1,1]
output_pref = os.path.join(dest, 'MOD11A1.A2014226.mosaic')
output_tif = os.path.join(dest, 'MOD11A1.A2014226.mosaic.tif')

First we'll initialize the mosaic object

In [ ]:
# the first parameter is a list with the original tiles,
# the second one is a list with the the subset to process,
# the last is the output format, in this case GeoTiff
mosaic = createMosaicGDAL(files, subset, 'GTiff')

At this point we create the xml file with the information of input data, and a multilayr GeoTiff file containing the mosaic of the choosen band

In [ ]:
mosaic.run(output_tif)

Finally we can also create a GDAL VRT file, a XML file containing the information about input data. This is really powerfull if you want convert it in different format or projection system

In [ ]:
mosaic.write_vrt(output_pref)

## Convert data

#### Single HDF file

In this first example we are going to convert an original MODIS HDF file using GDAL library

In [ ]:
from pymodis.convertmodis_gdal import convertModisGDAL
output_pref = os.path.join(dest, 'MOD11A1.A2014226.h18v04')

We are going to convert a single tile, with the 'subset' already used for mosaicking, the output resolution will be 1000 meters and the projection system will be EU-LAEA

In [ ]:
convertsingle = convertModisGDAL(hdfname=files[0], prefix=output_pref, subset=subset, res=1000, epsg=3035)
convertsingle.run()

#### VRT mosaic file

We created separated VRT file, one for each choosen subset. So first we collect the name of VRT files

In [ ]:
vrtfiles = glob.glob(os.path.join(dest, 'MOD11A1.A2014*.vrt'))
print vrtfiles

Now we can convert all the VRT files in a *``for``* loop 

In [ ]:
for f in vrtfiles:
    base = os.path.basename(f).replace('.vrt', '_vrt')
    output = os.path.join(dest, base)
    convertsingle = convertModisGDAL(hdfname=f, prefix=output, subset= [1,1,1,1], res=1000, epsg=3035, vrt=True)
    convertsingle.run_vrt_separated()

It will create 4 GeoTIFF files ready to be processed.

In [ ]:
vrttiffiles = glob.glob(os.path.join(dest, 'MOD11A1.A2014*_vrt.tif'))
print vrttiffiles

In [ ]:
from topng import tif2png
pngfile = tif2png(vrttiffiles[1])
Image(filename=pngfile)

### The tutorial is finished, you can find more info at <a href="http://www.pymodis.org/" targe="_blank">pyModis website</a>